In [3]:
# Ramsey King
# DSC 550 - Data Mining
# November 7, 2021
# Exercise 9.3

In [4]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2


data = pd.read_json('categorized-comments.jsonl', lines=True)
data.head(1)



,cat,txt
0,sports,Barely better than Gabbert? He was significant...


In [5]:
category_list = list(set(data['cat']))
category_list

['sports', 'video_games', 'science_and_technology']

In [7]:
def make_categorical(corpus_cat):
    cat_dictionary = {
        'science_and_technology': 1,
        'sports': 2,
        'video_games': 3,
    }

    return cat_dictionary.get(corpus_cat)

data['cat_num'] = data['cat'].apply(make_categorical)


1. Neural Network Classifier with Scikit

Using the multi-label classifier dataset (categorized-comments.jsonl), fit a neural network classifier using scikit-learn to predict the comment category. Use the code found in chapter 12 of the Applied Text Analysis with Python book as a guide, but you will need to modify the code for this dataset. Report the accuracy, precision, recall, F1-score, and confusion matrix.

In [8]:
stemmer = SnowballStemmer('english')
stop_words = stopwords.words('english')

data['processed_text'] = data['txt'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub('[^a-zA-Z]', " ", x).split() if i not in stop_words]).lower())
data['processed_text'][0]

'bare better gabbert he signific better year he th passer rate ahead guy like philip river carson palmer eli man tyrod taylor and worst surround offens cast leagu he without doubt better career backup list although jimmi g could potenti better riski kaep especi get one cheap year anyway still get rid much acquir'

We will get the scores required to satsify the exercise requirements:

In [11]:
X_train, X_test, y_train, y_test = train_test_split(data['processed_text'], data.cat_num, test_size=0.2)

pipeline = Pipeline([('vect', TfidfVectorizer(ngram_range=(1,2), stop_words='english', sublinear_tf=True)),
('chi', SelectKBest(chi2, k=10000)),
('clf', LinearSVC(C=1.0, penalty='l1', max_iter=3000, dual=False))
])

model = pipeline.fit(X_train, y_train)

vectorizer = model.named_steps['vect']
chi = model.named_steps['chi']
clf = model.named_steps['clf']

feature_names = vectorizer.get_feature_names_out()
feature_names = [feature_names[i] for i in chi.get_support(indices=True)]
feature_names = np.asarray(feature_names)

target_names = ['1', '2', '3']

print("Top 10 keywords per class: ")
for i, label in enumerate(target_names):
    top10 = np.argsort(clf.coef_[i])[-10:]
    print('%s: %s' % (label, " ".join(feature_names[top10])))

print("accuracy score: " + str(model.score(X_test, y_test)))

print(model.predict(['Tom Brady is the GOAT!']), '1 = Science and Tech', '2 = Sports', '3 = Video Games')
print(model.predict(['Super Mario Bros is the best ever']), '1 = Science and Tech', '2 = Sports', '3 = Video Games')



Top 10 keywords per class: 
1: sms com android nougat linkm xda whatsapp moto bootloop rcs allo
2: futbin sif wl clemson liverpool tailgat otw sbc klopp cfb
3: blizzard rubi mmr moto moto joycon amethyst dota muthead nintendo mut
accuracy score: 0.8561452974541617
[2] 1 = Science and Tech 2 = Sports 3 = Video Games
[3] 1 = Science and Tech 2 = Sports 3 = Video Games


In [12]:

# from transformer import TextNormalizer
# from reader import PickledReviewsReader

from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

# Path to postpreprocessed, part-of-speech tagged review corpus
cpath = data['tokenized_txt']

regressor = Pipeline([
    ('norm', data['normalized_txt']),
    ('tfidf', TfidfVectorizer()),
    ('ann', MLPRegressor(hidden_layer_sizes=[500,150], verbose=True))
])
regression_scores = train_model(cpath, regressor, continuous=True)

classifier = Pipeline([
    ('norm', data['normalized_txt']),
    ('tfidf', TfidfVectorizer()),
    ('ann', MLPClassifier(hidden_layer_sizes=[500,150], verbose=True))
])
classifer_scores = train_model(cpath, classifier, continuous=False)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().